In [8]:
# Cell 1: Import libraries and setup
import pandas as pd
import numpy as np
import os

# Create processed directory if it doesn't exist
os.makedirs('data/processed', exist_ok=True)

# Read the extracted CSV files
census_dfs = {
    1900: pd.read_csv('data/extracted/lakeland_1900_census.csv'),
    1920: pd.read_csv('data/extracted/lakeland_1920_census.csv'),
    1930: pd.read_csv('data/extracted/lakeland_1930_census.csv'),
    1940: pd.read_csv('data/extracted/lakeland_1940_census.csv'),
    1950: pd.read_csv('data/extracted/lakeland_1950_census.csv')
}


In [9]:
# Cell 2: Standardize column names
def standardize_columns(df, year):
    """Standardize column names to match database schema"""
    column_mapping = {
        'pk': 'source_pk',
        'dwelling number': 'dwelling_number',
        'dwelling': 'dwelling_number',
        'house_number': 'house_num',
        'relation_head': 'relation_to_head',
        'head_last': 'head_last_name',
        'head_first': 'head_first_name',
        'marital': 'marital_status',
        'birth_place': 'place_birth',
        'family': 'family_number'
    }
    
    df = df.copy()
    df = df.rename(columns=column_mapping)
    df['census_year'] = year
    return df

# Create standardized dataframes
standardized_dfs = {}
for year, df in census_dfs.items():
    standardized_dfs[year] = standardize_columns(df, year)

In [10]:
# Cell 3: Create census_records table
census_records_data = []
record_id_counter = 1

for year, df in standardized_dfs.items():
    census_records = pd.DataFrame({
        'record_id': range(record_id_counter, record_id_counter + len(df)),
        'census_year': df['census_year'].astype('Int64'),
        'source_pk': df['source_pk'].astype('Int64'),
        'ed': df['ed'] if 'ed' in df.columns else None,
        'page_number': df['page_number'] if 'page_number' in df.columns else None
    })
    record_id_counter += len(df)
    census_records_data.append(census_records)

census_records_table = pd.concat(census_records_data, ignore_index=True)

In [11]:
# Cell 4: Create persons table with unique IDs
def generate_person_id(year, index):
    return f"P{year}_{index:06d}"

persons_data = []
for year, df in standardized_dfs.items():
    persons = pd.DataFrame({
        'person_id': [generate_person_id(year, i) for i in range(len(df))],
        'first_name': df['first_name'],
        'last_name': df['last_name']
    })
    persons_data.append(persons)

persons_table = pd.concat(persons_data, ignore_index=True).drop_duplicates()

In [12]:
# Cell 5: Create locations table
locations_data = []
location_id_counter = 1

for year, df in standardized_dfs.items():
    locations = pd.DataFrame({
        'location_id': range(location_id_counter, location_id_counter + len(df)),
        'record_id': range(record_id_counter - len(df), record_id_counter),
        'street_name': df['street_name'] if 'street_name' in df.columns else None,
        'house_num': df['house_num'] if 'house_num' in df.columns else None,
        'build_num': df['build_num'] if 'build_num' in df.columns else None,
        'dwelling_number': df['dwelling_number'],
        'family_number': df['family_number']
    })
    location_id_counter += len(df)
    locations_data.append(locations)

locations_table = pd.concat(locations_data, ignore_index=True)

In [13]:
# Cell 6: Create families table
def generate_family_id(year, index):
    return f"F{year}_{index:06d}"

families_data = []
for year, df in standardized_dfs.items():
    # Group by dwelling and family number to create unique families
    group_cols = ['dwelling_number', 'family_number'] if 'dwelling_number' in df.columns else ['family_number']
    family_groups = df.groupby(group_cols)
    
    families = pd.DataFrame({
        'family_id': [generate_family_id(year, i) for i in range(len(family_groups))],
        'record_id': range(record_id_counter - len(df), record_id_counter),
        'location_id': range(location_id_counter - len(df), location_id_counter),
        'head_first_name': family_groups.first()['head_first_name'] if 'head_first_name' in df.columns else None,
        'head_last_name': family_groups.first()['head_last_name'] if 'head_last_name' in df.columns else None
    })
    families_data.append(families)

families_table = pd.concat(families_data, ignore_index=True)

Persons Table Sample:
      person_id first_name      last_name
0  P1900_000000       Ezra  Vanvalkenburg
1  P1900_000001     Hattie  Vanvalkenburg
2  P1900_000002    William  Vanvalkenburg
3  P1900_000003        Jay  Vanvalkenburg
4  P1900_000004     Jessie  Vanvalkenburg


In [14]:
# Cell 7: Create personal_attributes table
def clean_age(age):
    """Convert age to integer, handling fractions"""
    if pd.isna(age):
        return None
    try:
        return int(float(age))
    except (ValueError, TypeError):
        if isinstance(age, str) and '/' in age:
            num, denom = map(int, age.split('/'))
            return max(0, int(num / denom))
        return None

personal_attributes_data = []
attribute_id_counter = 1

for year, df in standardized_dfs.items():
    attributes = pd.DataFrame({
        'attribute_id': range(attribute_id_counter, attribute_id_counter + len(df)),
        'person_id': [generate_person_id(year, i) for i in range(len(df))],
        'record_id': range(record_id_counter - len(df), record_id_counter),
        'sex': df['sex'],
        'race': df['race'],
        'age': df['age'].apply(clean_age),
        'place_birth': df['place_birth'] if 'place_birth' in df.columns else None
    })
    attribute_id_counter += len(df)
    personal_attributes_data.append(attributes)

personal_attributes_table = pd.concat(personal_attributes_data, ignore_index=True)

In [15]:
# Cell 8: Create occupations table
occupations_data = []
occupation_id_counter = 1

for year, df in standardized_dfs.items():
    occupations = pd.DataFrame({
        'occupation_id': range(occupation_id_counter, occupation_id_counter + len(df)),
        'person_id': [generate_person_id(year, i) for i in range(len(df))],
        'record_id': range(record_id_counter - len(df), record_id_counter),
        'work': df['work'].replace({np.nan: None}),
        'business': df['business'].replace({np.nan: None}) if 'business' in df.columns else None
    })
    occupation_id_counter += len(df)
    occupations_data.append(occupations)

occupations_table = pd.concat(occupations_data, ignore_index=True)

Occupations Table Sample:
      person_id         work business  record_id  occupation_id
0  P1900_000000     Merchant     None          0              1
1  P1900_000001         None     None          1              2
2  P1900_000002  Bakes Bread     None          2              3
3  P1900_000003        Baker     None          3              4
4  P1900_000004    At School     None          4              5


In [16]:
# Cell 9: Create relationships table
relationships_data = []
relationship_id_counter = 1

for year, df in standardized_dfs.items():
    # Get family assignments from the families table
    family_assignments = df.groupby(['dwelling_number', 'family_number']).ngroup()
    relationships = pd.DataFrame({
        'relationship_id': range(relationship_id_counter, relationship_id_counter + len(df)),
        'person_id': [generate_person_id(year, i) for i in range(len(df))],
        'family_id': [generate_family_id(year, idx) for idx in family_assignments],
        'record_id': range(record_id_counter - len(df), record_id_counter),
        'relation_to_head': df['relation_to_head'].replace({np.nan: None})
    })
    relationship_id_counter += len(df)
    relationships_data.append(relationships)

relationships_table = pd.concat(relationships_data, ignore_index=True)


Relationships Table Sample:
      person_id     family_id relation_to_head  record_id  relationship_id
0  P1900_000000  F1900_000000             Head          0                1
1  P1900_000001  F1900_000001             Wife          1                2
2  P1900_000002  F1900_000002              Son          2                3
3  P1900_000003  F1900_000003              Son          3                4
4  P1900_000004  F1900_000004              Son          4                5


In [18]:
# Cell 10: Create families table with required columns
families_data = []

for year, df in standardized_dfs.items():
    families = pd.DataFrame({
        'family_id': df['family_id'],
        'record_id': df.index + 1,  # Start from 1
        'location_id': locations_table.loc[df.index, 'location_id'],  # Link to locations
        'head_first_name': df['head_first_name'] if 'head_first_name' in df.columns else None,
        'head_last_name': df['head_last_name'] if 'head_last_name' in df.columns else None
    }).drop_duplicates('family_id')
    families_data.append(families)

families_table = pd.concat(families_data, ignore_index=True)


In [72]:
# Cell 11: Create property_status table
property_status_data = []

for year, df in standardized_dfs.items():
    if 'owned_rented' in df.columns:
        property_status = pd.DataFrame({
            'person_id': df['person_id'],
            'owned_rented': df['owned_rented'],
            'record_id': df.index
        })
        property_status['property_id'] = range(1, len(property_status) + 1)
        property_status_data.append(property_status)

property_status_table = pd.concat(property_status_data, ignore_index=True)
print("Property Status Table Sample:")
print(property_status_table.head())

Property Status Table Sample:
      person_id owned_rented  record_id  property_id
0  P1900_000000            O          0            1
1  P1900_000001          NaN          1            2
2  P1900_000002          NaN          2            3
3  P1900_000003          NaN          3            4
4  P1900_000004          NaN          4            5


In [73]:
# Cell 12: Create marital_status table
marital_status_data = []

for year, df in standardized_dfs.items():
    if 'marital_status' in df.columns:
        marital = pd.DataFrame({
            'person_id': df['person_id'],
            'marital_status': df['marital_status'],
            'record_id': df.index
        })
        marital['marital_id'] = range(1, len(marital) + 1)
        marital_status_data.append(marital)

marital_status_table = pd.concat(marital_status_data, ignore_index=True)
print("Marital Status Table Sample:")
print(marital_status_table.head())

Marital Status Table Sample:
      person_id marital_status  record_id  marital_id
0  P1900_000000              M          0           1
1  P1900_000001              M          1           2
2  P1900_000002              S          2           3
3  P1900_000003              S          3           4
4  P1900_000004              S          4           5


In [74]:
# Cell 13: Data cleaning function
def clean_dataframe(df):
    """Clean DataFrame by handling NaN values and data types"""
    # Replace NaN with None for string columns
    object_columns = df.select_dtypes(include=['object']).columns
    for col in object_columns:
        df[col] = df[col].replace({np.nan: None})
    
    # Handle numeric columns
    numeric_columns = df.select_dtypes(include=['float64', 'int64']).columns
    for col in numeric_columns:
        if col not in ['location_id', 'record_id', 'attribute_id', 'occupation_id', 'relationship_id', 'property_id', 'marital_id']:
            df[col] = df[col].replace({np.nan: None})
    
    return df

In [75]:
# Cell 14: Export with proper data types
for table_name, df in tables.items():
    # Ensure all string columns are properly handled
    for col in df.select_dtypes(include=['object']).columns:
        df[col] = df[col].replace({np.nan: None})
    
    # Ensure all numeric columns are properly typed
    numeric_cols = {
        'census_records': ['census_year', 'source_pk', 'record_id'],
        'locations': ['location_id', 'record_id'],
        'personal_attributes': ['age', 'attribute_id', 'record_id'],
        'occupations': ['occupation_id', 'record_id'],
        'families': ['record_id', 'location_id'],
        'relationships': ['relationship_id', 'record_id'],
        'property_status': ['property_id', 'record_id'],
        'marital_status': ['marital_id', 'record_id']
    }
    
    if table_name in numeric_cols:
        for col in numeric_cols[table_name]:
            if col in df.columns:
                df[col] = df[col].astype('Int64')
    
    # Export to CSV
    output_path = f'data/processed/{table_name}.csv'
    df.to_csv(output_path, index=False)
    print(f"Exported {table_name} to {output_path}")

Exported census_records to data/processed/census_records.csv
Exported locations to data/processed/locations.csv
Exported persons to data/processed/persons.csv
Exported personal_attributes to data/processed/personal_attributes.csv
Exported occupations to data/processed/occupations.csv
Exported families to data/processed/families.csv
Exported relationships to data/processed/relationships.csv
Exported property_status to data/processed/property_status.csv
Exported marital_status to data/processed/marital_status.csv


In [76]:
# Cell 14: Verify data integrity
for table_name, df in tables.items():
    print(f"\n{table_name} Summary:")
    print(f"Number of rows: {len(df)}")
    print(f"Columns: {df.columns.tolist()}")
    print(f"Sample of first few rows:")
    print(df.head())
    print("-" * 50)


census_records Summary:
Number of rows: 797
Columns: ['census_year', 'source_pk', 'ed', 'page_number', 'record_id']
Sample of first few rows:
   census_year  source_pk    ed page_number  record_id
0         1900          1  None        None          1
1         1900          2  None        None          2
2         1900          3  None        None          3
3         1900          4  None        None          4
4         1900          5  None        None          5
--------------------------------------------------

locations Summary:
Number of rows: 797
Columns: ['street_name', 'house_num', 'build_num', 'dwelling_number', 'family_number', 'location_id', 'record_id']
Sample of first few rows:
  street_name house_num build_num dwelling_number family_number  location_id  \
0        None      None      None              32            32            1   
1        None      None      None              32            32            2   
2        None      None      None              32      